In [1]:
import cooler
import numpy as np
import pandas as pd
import anndata
from glob import glob

In [12]:
ct = 'Oligo_NN'

In [2]:
indir = 'dataset/domain/'
res = 25000

In [3]:
chrom_size_path = '/ref/m3C/mm10.main.nochrM.nochrY.chrom.sizes'
chrom_sizes = cooler.read_chromsizes(chrom_size_path, all_names=True)

In [16]:
meta_file = '/data/metadata/240104_m3C_META.csv'
metadata = pd.read_csv(meta_file, index_col = 0)
metadata['AgingMajorType'] = metadata['AgingMajorType'].apply(lambda x: x.replace(' ','_').replace('/',''))
metadata = metadata[['AgingMajorType','Age','Replicate']]

In [17]:
tad_path_list = glob(f'{indir}/*.boundary.h5ad')
print(len(tad_path_list))

3


In [18]:
sc_border = []
for xx in tad_path_list:
    sc_border.append(anndata.read_h5ad(xx))
    
sc_border = anndata.AnnData.concatenate(*sc_border, index_unique=None)
sc_border

AnnData object with n_obs × n_vars = 900 × 105362
    obs: 'batch'
    var: 'chrom', 'start', 'end'

In [19]:
sc_border.X = sc_border.X.astype(int)

In [20]:
sc_border = sc_border[sc_border.obs.index.isin(metadata.index)].copy()
sc_border = sc_border[:, ~sc_border.var['chrom'].isin(['chrX','chrY','chrM','chrL'])].copy()

In [21]:
sc_border.obs = metadata.loc[sc_border.obs.index]
sc_border.raw = sc_border.copy()

In [22]:
sc_border.write_h5ad(f'cell_{sc_border.shape[0]}_boundary.h5ad')

In [26]:
sc_border.obs['AgingMajorType.Age'] = sc_border.obs['AgingMajorType'].astype(str) + '.' + sc_border.obs['Age'].astype(str)

In [29]:
leg = sc_border.obs['AgingMajorType.Age'].unique()

In [31]:
bound_count_ct = []
for xx in leg:
    bound_count_ct.append(sc_border.raw.X[sc_border.obs['AgingMajorType.Age']==xx].getnnz(axis=0))

bound_count_ct = pd.DataFrame(bound_count_ct, index=leg, 
                              columns=sc_border.raw.var['chrom'].astype(str) + '_' + (sc_border.raw.var['start'] // res).astype(str))
cell_count_ct = sc_border.obs['AgingMajorType.Age'].value_counts().loc[leg]

In [35]:
bound_count_ct.to_hdf(f'{ct}_boundcount.hdf', key='data')
cell_count_ct.to_csv(f'{ct}_cellcount.csv.gz')